In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [45]:
ref_df = pd.read_csv('./data/3775610.csv')

In [46]:
# resturture the data files (Pandas)

def restructure_csv_files(directory, ref_df, new_directory):
    files = [f for f in os.listdir(directory) if f.endswith('.csv')]

    if not files:
        print("No CSV files found in the directory.")
        return

    ref_columns = ref_df.columns.tolist()

    for file in files:
        df = pd.read_csv(os.path.join(directory, file))

        if df.columns.tolist() != ref_columns:
            # Add missing columns
            df = df.loc[:, [i for i in ref_columns if i in df.columns]]
            df.loc[:, [i for i in ref_columns if i not in df.columns]] = None

            # Reorder columns and save the DataFrame
            df = df.loc[:, ref_columns]
            df.to_csv(os.path.join(new_directory, file), header=True, index=False, sep=',')

# Call the function with your directory and reference DataFrame
restructure_csv_files('./data/', ref_df, './processed data/')

In [47]:
"""def check_csv_structure(directory):
    files = [f for f in os.listdir(directory) if f.endswith('.csv')]
    if not files:
        print("No CSV files found in the directory.")
        return []

    # Read the first file to get the reference structure
    # ref_df = pd.read_csv(os.path.join(directory, files[0]))
    ref_columns = ref_df.columns.tolist()

    different_structure_files = []
    for file in files[1:]:
        df = pd.read_csv(os.path.join(directory, file), sep=',')
        if df.columns.tolist() != ref_columns:
            different_structure_files.append(file)

    return different_structure_files

# Call the function with your directory
different_files = check_csv_structure('./processed data/')
print(different_files)"""

'def check_csv_structure(directory):\n    files = [f for f in os.listdir(directory) if f.endswith(\'.csv\')]\n    if not files:\n        print("No CSV files found in the directory.")\n        return []\n\n    # Read the first file to get the reference structure\n    # ref_df = pd.read_csv(os.path.join(directory, files[0]))\n    ref_columns = ref_df.columns.tolist()\n\n    different_structure_files = []\n    for file in files[1:]:\n        df = pd.read_csv(os.path.join(directory, file), sep=\',\')\n        if df.columns.tolist() != ref_columns:\n            different_structure_files.append(file)\n\n    return different_structure_files\n\n# Call the function with your directory\ndifferent_files = check_csv_structure(\'./processed data/\')\nprint(different_files)'

In [54]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import *
import ast
import os

os.environ['PYSPARK_SUBMIT_ARGS'] = '--driver-class-path /home/lymf/mysql-connector-j-8.1.0.jar pyspark-shell'

# Create PySpark SparkSession
spark = SparkSession.builder.appName("pandas to spark").getOrCreate()

# Define a UDF to convert strings to lists using ast.literal_eval
# def literal_eval_udf(x):
#     return ast.literal_eval(x) if x is not None else x

# literal_eval_udf = udf(literal_eval_udf, ArrayType(StringType()))

# Define a UDF to extract the last name from the 'player' column
def get_last_name(x):
    return x.split(' ')[-1] if x is not None else x

get_last_name_udf = udf(get_last_name, StringType())

# Read all CSV files in a folder into a DataFrame
df = spark.read.format('csv').option('header', 'true').option('quote', '\"').option('escape', '\"').load("./processed data/*.csv")

# df = df.withColumn('player_last_name', get_last_name_udf(df['player']))

In [56]:
from pyspark.sql import SparkSession

# Database connection URL
url = "jdbc:mysql://localhost:3306/ftdata"

# Properties
properties = {
    "user": "debian-sys-maint",
    "password": "vMCs6xaR5jYNdv3u",
    "driver": "com.mysql.jdbc.Driver"
}

df.write.jdbc(url=url, table="match_events", mode="overwrite", properties=properties)


Loading class `com.mysql.jdbc.Driver'. This is deprecated. The new driver class is `com.mysql.cj.jdbc.Driver'. The driver is automatically registered via the SPI and manual loading of the driver class is generally unnecessary.
